In [49]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options

import pandas as pd
import numpy as np

import random
import re
import time

- 006751 전체대학
- 126897 KU융합과학기술원
- 127662 KU혁신공유대학
- 105271 건축대학
- 105541 경영대학
- 103041 공과대학
- 100251 교무처
- 127772 국제대학
- 126940 국제처
- 122046 글로벌융합대학
- 126843 대학교육혁신원
- 102761 문과대학
- 127425 부동산과학원
- 104951 사범대학
- 127119 사회과학대학
- 103781 상경대학
- 126841 상허교양대학
- 126896 상허생명과학대학
- 104351 생명환경과학대학
- 105061 수의과대학
- 122045 예술디자인대학
- 105121 이과대학
- 127304 혁신공유대학(글로컬)

In [ ]:
# 초기값
syllabus_list = []  # 각 강의계획서 내용이 데이터프레임 형태로 차례로 저장되는 빈 리스트 정의
debug_trial = 0     # 파싱에 실패하는 횟수

# 강의계획서 항목 이름 딕셔너리
menu_dict = {1: '일반사항', 2: '평가비율', 3: '강의교재', 4: '강의과제', 5: '주별강의계획서'}

# ================================================================================================

# 크롬 옵션 설정
options = Options()
options.add_argument("--start-maximized")  # 창 최대화
options.add_argument("--incognito")  # 시크릿 모드로 실행
options.add_argument("--disable-extensions")  # 확장 프로그램 비활성화

# 크롬 드라이버 사용
driver = webdriver.Chrome(options=options)
driver.get('https://sugang.konkuk.ac.kr/sugang/jsp/search/searchMainOuter.jsp')

# 메인 창 핸들 저장
main_window_handle = driver.current_window_handle

# 드롭다운 로딩 대기
time.sleep(2)

# 연도 반복
for year in ['2022', '2023', '2024']:  # 텍스트로 지정해야 합니다.
    print(f'Current Year: {year}')

    # 단과대학 번호 목록 (자기설계전공, 현장실습, 학부논문작성연습, 실감미디어, 글로컬 제외)
    univ_dict = {'126897': 'KU융합과학기술원', '105271': '건축대학', '105541': '경영대학', '103041': '공과대학',
                 '102761': '문과대학', '127425': '부동산과학원', '104951': '사범대학', '127119': '사회과학대학',
                 '126896': '상허생명과학대학', '105061': '수의과대학', '122045': '예술디자인대학', '105121': '이과대학'}

    # 드롭다운 - 연도
    select_year = driver.find_element(By.NAME, 'pYear')
    Select(select_year).select_by_value(year)

    # 적응 시간
    time.sleep(np.round(random.uniform(0.5, 1), 3))

    # 드롭다운 - 학기 (2학기만 확인)
    select_semester = driver.find_element(By.NAME, 'pTerm')
    Select(select_semester).select_by_value('B01012')

    # 적응 시간
    time.sleep(np.round(random.uniform(0.5, 1), 3))

    # 2022, 2023년에는 글로벌융합대학 추가, 2024년에는 국제대학 추가
    if year != '2024':
        univ_dict['122046'] = '글로벌융합대학'
    else:
        univ_dict['127772'] = '국제대학'

    # 단과대 반복
    for univ_idx, univ_number in enumerate(univ_dict.keys(), 1):
        print(f'  {univ_idx}. Collecting {univ_dict[univ_number]} . . .')
        
        # 드롭다운 - 대학
        select_univ = driver.find_element(By.NAME, 'pUniv')
        Select(select_univ).select_by_value(univ_number)

        # 글로벌융합대학일 경우 융합인재학부 지정 필요
        if univ_number == '122046':
            select_department = driver.find_element(By.NAME, 'pSustMjCd')  # 드롭다운 - 학과
            Select(select_department).select_by_value('122410')
            time.sleep(0.5)
        
        # 조회 버튼 클릭
        search_button = driver.find_element(By.CLASS_NAME, 'btn-sub')
        search_button.click()
    
        # 적응 시간
        time.sleep(np.round(random.uniform(0.5, 1), 3))
        
        # 과목번호 버튼 - 클래스명이 'btn-sm btn-sub'인 모든 버튼 가져오기
        number_buttons = driver.find_elements(By.CLASS_NAME, 'btn-sm.btn-sub')
    
        # 적응 시간
        time.sleep(np.round(random.uniform(1, 2), 3))
        
        # 각 버튼 클릭 및 팝업 처리
        for button in number_buttons:
            # 과목번호 추출하기
            onclick_value = button.get_attribute('onclick')
            button_number = re.search(r"\('(\d+)'\)", onclick_value).group(1)  # 정규표현식으로 숫자 부분만 추출
            
            # 클릭할 강의계획서의 내용을 저장할 딕셔너리 정의
            syllabus_dict = {'연도': year, '단과대학': univ_dict[univ_number], '과목번호': button_number}
            
            # 클릭할 버튼이 화면에 보이도록 스크롤
            driver.execute_script("arguments[0].scrollIntoView(true);", button)
            time.sleep(0.5)

            # 강의계획서 클릭
            button.click()
            
            # 팝업창 대기 (팝업이 뜨면 창 개수가 증가)
            WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))
            
            # 팝업창 핸들 탐색 및 전환
            for handle in driver.window_handles:
                if handle != main_window_handle:
                    driver.switch_to.window(handle)  # 팝업창으로 전환
                    break
            
            # 팝업창에서 원하는 데이터 크롤링
            try:
                # 팝업창에서 특정 요소 로드 대기
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'tbl-detail'))
                )
        
                # HTML 소스 가져오기
                page_source = driver.page_source
    
                # BeautifulSoup으로 HTML 파싱
                soup = BeautifulSoup(page_source, 'html.parser')
                
                # <table class="tbl-detail"> 안의 <td> 태그 텍스트 추출
                tables = soup.find_all('table', class_='tbl-detail')  # 모든 <table class="tbl-detail"> 요소 찾기
                
                # 모든 테이블의 <td> 태그 텍스트를 추출
                for table_idx, table in enumerate(tables, 1):  # 각 테이블에 대해 반복
                    # 현재 테이블의 모든 <td> 태그 가져오기
                    td_elements = table.find_all('td')
                    td_texts = {idx: td.text.strip() for idx, td in enumerate(td_elements, 1)}  # 텍스트만 추출하고 양쪽 공백 제거               
                    syllabus_dict[menu_dict[table_idx]] = td_texts  # 일반사항 ~ 주별강의계획서 내용 입력
                
            except Exception as e: 
                # 오류 메시지 출력
                print('HTML 파싱 실패', e)
    
                # 팝업창 스크린샷 촬영
                debug_trial += 1
                driver.save_screenshot(f"debug_screenshot_{debug_trial}.png")
    
            # 강의계획서 업데이트
            syllabus_list.append(pd.DataFrame([syllabus_dict]))
            
            # 팝업창 닫기
            driver.close()
            driver.switch_to.window(main_window_handle)  # 메인 창으로 복귀

# 크롤링 완료
driver.quit()

# 모든 강의계획서를 데이터프레임으로 한번에 병합
result_df = pd.concat(syllabus_list, ignore_index=True)
print("크롤링이 모두 완료되었습니다!")

In [59]:
result_df.head()

,연도,단과대학,과목번호,일반사항,평가비율,강의교재,강의과제,주별강의계획서
0,2022,KU융합과학기술원,3854,"{1: '3854', 2: '에너지공학실험1', 3: '', 4: '스스로 학습활동...","{1: '출석률', 2: '10', 3: '100', 4: '공개', 5: '중간고...",{1: '강의교재 정보가 존재하지 않습니다.'},"{1: '실험결과를 바탕으로 한 해석', 2: '20191211', 3: ''}","{1: '08/29 ~ 09/03', 2: 'Introduction', 3: 'Ba..."
1,2022,KU융합과학기술원,3855,"{1: '3855', 2: '에너지전공심화실습1', 3: '', 4: '다양한 정보...","{1: '출석률', 2: '20', 3: '100', 4: '공개', 5: '중간고...","{1: '주교재', 2: 'PDF file', 3: '', 4: '', 5: ''}","{1: 'Summary for Literature survey', 2: '20211...","{1: '08/29 ~ 09/03', 2: 'Introduction', 3: 'In..."
2,2022,KU융합과학기술원,3856,"{1: '3856', 2: '기기분석', 3: '', 4: '스스로 학습활동에 대한...","{1: '출석률', 2: '10', 3: '100', 4: '공개', 5: '중간고...","{1: '부교재', 2: '기기분석', 3: '정맹준', 4: '드림플러스', 5:...","{1: '나노소재 물성 분석에 대한 최신 기술 동향', 2: '20211206', ...","{1: '08/29 ~ 09/03', 2: '강의 개요', 3: '기기분석학 소개'..."
3,2022,KU융합과학기술원,3857,"{1: '3857', 2: '전기화학특론', 3: '', 4: '스스로 학습활동에 ...","{1: '출석률', 2: '15', 3: '15', 4: '공개', 5: '중간고사...","{1: '주교재', 2: '전기화학 2판', 3: '오승모', 4: '자유아카데미'...","{1: '강의 주제 관련 문제 풀이', 2: '20201024', 3: ''}","{1: '08/29 ~ 09/03', 2: 'Electrchemistry Funda..."
4,2022,KU융합과학기술원,3858,"{1: '3858', 2: '에너지공학종합설계', 3: '', 4: '상대방과 효과...","{1: '출석률', 2: '10', 3: '100', 4: '공개', 5: '중간고...",{1: '강의교재 정보가 존재하지 않습니다.'},{1: 'Energy-related materials or device design...,"{1: '08/29 ~ 09/03', 2: 'Introduction to capst..."


In [58]:
result_df.shape

(451, 8)

In [ ]:
result_df.to_csv('전선_크롤링.csv',index=False)